In [1]:
# Import Modules
import numpy as np
import pandas as pd
import scipy as sp
import dionysus as d
import matplotlib.pyplot as plt
import pecan as pc
import sys
#from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from sklearn.metrics.pairwise import euclidean_distances
import networkx as nx
import matplotlib.pyplot as plt
import argparse
import matplotlib.collections
import matplotlib.lines
import matplotlib.animation as animation
import gudhi as gd  
import collections
import sys 
import os
import re
#sys.path.append(os.path.abspath(""))
from basic_functions import *
from visualizations import *
from HomogeneousFiltration import *
from InhomogeneousFiltration import *

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Load data

###### double_annulus

In [2]:
experiment_name='double_annulus_128'
experiment_path='data/'
e= 0.0089   

###### petals

In [3]:
experiment_name='petals_128'
experiment_path='data/'
e= 0.1094   

###### rings

In [ ]:
experiment_name='rings_1_07_003'
experiment_path='data/'
e= 0.0999   

In [ ]:
experiment_name='rings_1_025_002_08'
experiment_path='data/'
e= 0.0958   

###### connected rings

In [ ]:
n_samples=100
n_connections=0
n_per_connection=10

experiment_name='connected_2_rings_'+str(n_samples)+'_'+str(n_connections)+'_'+str(n_per_connection)
experiment_path='data/'

e=0.1316

In [ ]:
n_samples=100
n_connections=3
n_per_connection=10

experiment_name='connected_2_rings_'+str(n_samples)+'_'+str(n_connections)+'_'+str(n_per_connection)
experiment_path='data/'

e=0.0859

In [ ]:
n_samples=100
n_connections=3
n_per_connection=20

experiment_name='connected_2_rings_'+str(n_samples)+'_'+str(n_connections)+'_'+str(n_per_connection)+'_'
experiment_path='data/'

e=0.0637

#### Select mode

In [4]:
kernel='gaussian_aniso'
mode='resting'
b=1   

# Mono-filtration over power of τ at fix weight

### Initialize

In [5]:
t=0                        # int: diffusion time step t 
tau_max=25                 # int: maximum power of tau - filtration from 0 ≤ tau ≤ tau_max
version='inhomogeneous'      # 'homogeneous' or 'inhomogeneous'

In [6]:
if version=='homogeneous' :
    PowerFiltration=OperatorPowerFiltrationHomogeneous(experiment_path,experiment_name,mode,kernel,e,b,t)
if version=='inhomogeneous' :
    PowerFiltration=OperatorPowerFiltrationInhomogeneous(experiment_path,experiment_name,mode,kernel,e,b,t)

### Weights

In [7]:
""" Compute all edge weights """

EdgeWeights_=PowerFiltration.compute_edge_weights(tau_max)

In [1]:
""" Visualization of edge weights """

" parameters"
EdgeWeights=EdgeWeights_            # Edge weights to be visualized
plot_weight_thresholds=None         # None or List of floats for horizontal lines in the plot
scale_='linear'                     # scale of the y-axis: 'linear' or 'log'
save_=True                          # save plot: True or False
save_path='results/plots'           # path to saved plot (subdirectories will be created automatically)

plot_weights_during_filtration(EdgeWeights,experiment_name,e,b,kernel,mode,version,t,W_2=None,W_3=None,scale=scale_,thresholds=plot_weight_thresholds,save=save_,save_path=save_path,label_1='',label_2='',label_3='')

NameError: name 'EdgeWeights_' is not defined

In [2]:
""" Filtration """

" parameters"
tau_max=10              # int: maximum power of tau - filtration from 0 ≤ tau ≤ tau_max
weight_threshold=0.035  # float: weight threshold for the filtration
dist_threshold=1e-3     # float: two points which are closer than dist_threshold are merged if merge==True
merge=False              # True or False: two points which are closer than dist_threshold are merged if merge==True
max_dimension=2         # int: maximal dimension simplex
plot_embedding='fix'    # 'fix' or 'var': Plot embedding of simplicial complex at fix t=0 or variable at different diffusion times
show_fig_=True          # True or False: If True, show figures in Notebook
save_fig_=True          # True or False: If True, save figures under save_path
save_path='results/plots'  #/simplicial_complex' 

betti_tensor, simplices_tensor,BarCodes=PowerFiltration.compute_filtration(weight_threshold,dist_threshold,tau_max, max_dimension,plot_embedding,merge,show_fig=show_fig_,save_fig=save_fig_,save_path=save_path)


NameError: name 'PowerFiltration' is not defined

In [1]:
save_path='results/plots'

PowerFiltration.plot_betti(betti_tensor,merge,weight_threshold,dist_threshold,scale='linear',show_fig=True,save_fig=True,save_path=save_path)

NameError: name 'PowerFiltration' is not defined

In [ ]:
save_path='results/plots'

PowerFiltration.plot_num_simplices(simplices_tensor,merge,weight_threshold,dist_threshold,scale='linear',show_fig=True,save_fig=True,save_path=save_path)

In [ ]:
save_path='results/plots'

persistence(BarCodes,experiment_name,e,b,t,kernel,mode,version,merge,dist_threshold,weight_threshold,save_path,save=True)

# Bifiltration

In [ ]:
""" Bifiltration over tau and weight threshold """

" parameters"
weight_threshold_steps=np.array([0.05,0.045,0.04,0.035,0.03,0.025,0.02,0.015]) # list of float: filtration steps over weight  weight_threshold_steps=np.array([0.07,0.0675,0.065,0.0625,0.06,0.0575,0.055,0.0525,0.05,0.0475 ,0.045,0.0425,0.04,0.0375,0.035,0.0325,0.03,0.0275,0.025,0.0225,0.02,0.0175,0.015,0.0125,0.01,0.0075])  #
tau_max=20              # int: filtration steps over tau - filtration from 0 ≤ tau ≤ tau_max
dist_threshold=1e-3     # float: two points which are closer than dist_threshold are merged if merge==True
merge=True              # True or False: two points which are closer than dist_threshold are merged if merge==True
max_dimension=2         # int: maximal dimension simplex
plot_embedding='var'    # 'fix' or 'var': Plot embedding of simplicial complex at fix t=0 or variable at different diffusion times
show_fig_=False          # True or False: If True, show figures in Notebook
save_fig_=False          # True or False: If True, save figures under save_path
save_path='results/plots'  #/simplicial_complex' 



betti_tensor_filtration=np.zeros((len(weight_threshold_steps),tau_max,3))
simplices_tensor_filtration=np.zeros((len(weight_threshold_steps),tau_max,3))
BarCodes_tensor=[]
for z in range(0,len(weight_threshold_steps)):
    weight_threshold=weight_threshold_steps[z]
    print(weight_threshold)
    betti_tensor, simplices_tensor,BarCodes=PowerFiltration.compute_filtration(weight_threshold,dist_threshold,tau_max, max_dimension,plot_embedding,merge,show_fig=show_fig_,save_fig=save_fig_,save_path=save_path)
    simplices_tensor_filtration[z]=simplices_tensor
    betti_tensor_filtration[z]=betti_tensor
    BarCodes_tensor.append(BarCodes)
    print()

In [ ]:
w_min=np.min(weight_threshold_steps)
w_max=np.max(weight_threshold_steps)

save_path='results/plots'  #/simplicial_complex'
save_fig=True

bifiltration_plot_betti(betti_tensor_filtration,w_min,w_max,version,experiment_name,e,b,t,kernel,mode,weight_threshold_steps,merge,dist_threshold,save_fig,save_path)


In [ ]:
w_min=np.min(weight_threshold_steps)
w_max=np.max(weight_threshold_steps)

save_path='results/plots'

bifiltration_plot_simplices(betti_tensor_filtration,w_min,w_max,version,experiment_name,e,b,t,kernel,mode,weight_threshold_steps,merge,dist_threshold,save_fig,save_path)

In [ ]:
save_path='results/plots'
for u in range(0,len(BarCodes_tensor)):
    print(r'$p_{max}=$',weight_threshold_steps[u])
    BarCodes_=BarCodes_tensor[u]
    persistence(BarCodes_,experiment_name,e,b,t,kernel,mode,version,merge,dist_threshold,weight_threshold,save_path,save=True)
